# Ranges

`chame` brings ranges operations the scverse ecosystem.

> Ranges functionality is currently an experimental feature with its current implementation based on the [bioframe](https://github.com/open2c/bioframe) library.

In the context of genomics, _genomic ranges_ are defined using coordinates for a specific chromosome and refer to contiguous DNA regions. Some formal introduction into the topic can be found in [Lawrence et al., 2013](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003118) as well as in the vignettes of the [GenomicRanges Bioconductor package](https://bioconductor.org/packages/devel/bioc/vignettes/GenomicRanges/inst/doc/GenomicRangesIntroduction.html). Typical operations on genomic ranges are also illustrated [in the bedtools documentation](https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html).

## Prepare data

In [1]:
import numpy as np
import pandas as pd

import mudata

Load ATAC modality from a multimodal object:

In [2]:
from pathlib import Path
data = Path("../../../muon-tutorials/data")

In [3]:
adata = mudata.read(str(data / "brain3k_processed.h5mu/atac"))
mdata = mudata.read(str(data / "brain3k_processed.h5mu"))

Import `chame`:

In [4]:
import chame as ch

## Filter by genomic ranges

Typically AnnData/MuData objects are subsetted (or _sliced_) along the feature dimension using column indices or column names:

In [5]:
adata[:,["chr1:817893-818459", "chr1:818576-819294", "chr1:827064-827944"]]

View of AnnData object with n_obs × n_vars = 2821 × 3
    obs: 'n_genes_by_counts', 'total_counts', 'nucleosome_signal', 'tss_score', 'leiden', 'celltype'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'atac', 'celltype_colors', 'files', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'lognorm'
    obsp: 'connectivities', 'distances'

Sometimes features are defined in some coordinate system such as a linear _genome sequence_ — for instance in assays that measure chromatin accessibility, transcription factor occupancy, DNA methylation. In fact, genes also have a property defining their location in the DNA, even though that's something frequently ignored in transcriptomics analysis pipelines.

In these cases, it is natural to subset the object based on a set of coordinates. This is the interface that `chame` implements:

In [6]:
ch.pp.filter_var_by_ranges(adata, "chr1:900000-1000000")

View of AnnData object with n_obs × n_vars = 2821 × 12
    obs: 'n_genes_by_counts', 'total_counts', 'nucleosome_signal', 'tss_score', 'leiden', 'celltype'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'atac', 'celltype_colors', 'files', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'lognorm'
    obsp: 'connectivities', 'distances'

It uses genomic intervals from the `.var` table. A single interval can be defined as

- a single column, e.g. `"interval"` (default, split into three columns during subsetting), or
- a set of three columns, e.g. `("Chromosome", "Start", "End")` (used by default).



In [7]:
ch.pp.filter_var_by_ranges(mdata, "chr1:900000-1000000")

View of MuData object with n_obs × n_vars = 2821 × 23
  var:	'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
  obsm:	'X_mofa', 'X_mofa_umap', 'X_umap', 'X_wnn_umap'
  varm:	'LFs'
  obsp:	'connectivities', 'distances', 'wnn_connectivities', 'wnn_distances'
  2 modalities
    atac:	2821 x 12
      obs:	'n_genes_by_counts', 'total_counts', 'nucleosome_signal', 'tss_score', 'leiden', 'celltype'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
      uns:	'atac', 'celltype_colors', 'files', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'counts', 'lognorm'
      obsp:	'connectivities', 'distances'
    rna:	2821 x 11
      obs:	'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'celltype'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
      uns:	'celltype_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'counts', 'lognorm'
      obsp:	'connectivities', 'distances'

Subsetting works on multiple genomic regions as well:

In [8]:
ch.pp.filter_var_by_ranges(adata, [f"chr{c}:900000-1000000" for c in range(22)])

View of AnnData object with n_obs × n_vars = 2821 × 145
    obs: 'n_genes_by_counts', 'total_counts', 'nucleosome_signal', 'tss_score', 'leiden', 'celltype'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'atac', 'celltype_colors', 'files', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'lognorm'
    obsp: 'connectivities', 'distances'

When a collection of genomic ranges is provided, it is likely to be represented as a NumPy array or a Pandas DataFrame, e.g.

In [9]:
query_ranges = pd.DataFrame({
    "Chromosome": ["chr1", "chr1", "chr2"],
    "Start": [1_000_000, 2_000_000, 3_000_000],
    "End": [1_100_000, 2_100_000, 3_100_000],
})
query_ranges

,Chromosome,Start,End
0,chr1,1000000,1100000
1,chr1,2000000,2100000
2,chr2,3000000,3100000


In [10]:
ch.pp.filter_var_by_ranges(adata, ranges=query_ranges)

View of AnnData object with n_obs × n_vars = 2821 × 36
    obs: 'n_genes_by_counts', 'total_counts', 'nucleosome_signal', 'tss_score', 'leiden', 'celltype'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'atac', 'celltype_colors', 'files', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'lognorm'
    obsp: 'connectivities', 'distances'

**Subsetting MuData**

Subsetting using genomic ranges works in the same way for MuData objects:

In [11]:
ch.pp.filter_var_by_ranges(mdata, [f"chr{c}:900000-1000000" for c in range(22)])

View of MuData object with n_obs × n_vars = 2821 × 207
  var:	'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
  obsm:	'X_mofa', 'X_mofa_umap', 'X_umap', 'X_wnn_umap'
  varm:	'LFs'
  obsp:	'connectivities', 'distances', 'wnn_connectivities', 'wnn_distances'
  2 modalities
    atac:	2821 x 145
      obs:	'n_genes_by_counts', 'total_counts', 'nucleosome_signal', 'tss_score', 'leiden', 'celltype'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
      uns:	'atac', 'celltype_colors', 'files', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'counts', 'lognorm'
      obsp:	'connectivities', 'distances'
    rna:	2821 x 62
      obs:	'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'celltype'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
      uns:	'celltype_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'counts', 'lognorm'
      obsp:	'connectivities', 'distances'